# Energy Management Example

## Narrative

Your company must decide how to mix the various forms of available energy:

- Nuclear: only ignition cost, generates constant energy
- Wind: only cost per kw, but not constant
- Gas: ignition cost each period plus cost per kw, you have to decide if and how much to produce, there exist an upper bound
- External purchase: buy energy from external providers but at a very high price

Energy demand is variable.



\begin{equation*}
\begin{aligned}
&\text{minimize} \sum_{\text{scenarios } s} \sum_{\text{periods } t} (&\text{nuclear-activation-cost}[s] +& \\
&& \text{gas-activation-cost[t, s]} +& \\
&& \text{gas-production}[t, s] +&\\
&& \text{wind-production}[t, s] +&\\
&& \text{external-purchase}[t,s] +&\\
&&& ) \times \text{probability}[s]
\end{aligned}
\end{equation*}



## Libraries

In [21]:
import numpy as np
import random
import pyomo.environ as pyo

BIG_M = 10000



## Cost Function Approximation


### Data

In [22]:
num_step = 3
scenarios_by_step = 6

all_scenarios = list(range(1, scenarios_by_step**num_step+1))


high_wind_probability = .6
low_wind_probability = .4

low_demand_probability = .3
medium_demand_probability = .4
high_demand_probability = .3


demands = {1: [17000, 12000, 10000],
           2: [12000, 10000, 80000],
           3: [18000, 15000, 90000]}



data = {}

data.update({'S' : all_scenarios})
data.update({'T' : list(range(1, num_step+1))})


non_ant_sets = {}
for t in range(num_step):
    elements = scenarios_by_step**(num_step-t)
    groups = scenarios_by_step**(t)
    non_ant_sets.update({t+1 : [ all_scenarios[i:i+elements] for i in range(0, groups) ]})

data.update({'NonAnt' : non_ant_sets})


data.update({'cost_activate_nuc' : {None: 30000}})
data.update({'power_nuc' : {None: 15000}})

data.update({'cost_activate_gas' : {None: 50}})
data.update({'cost_kw_gas' : {None: 3}})
data.update({'max_power_gas' : {None: 5000}})

data.update({'cost_kw_wind' : {None: 2}})

data.update({'power_wind_high' : {None: 7000}})
data.update({'power_wind_low' : {None: 2000}})
data.update({'cost_kw_ext' : {None: 10}})

wind_high_param = {}
for t in range(num_step):
    elements = int(scenarios_by_step**(num_step-t)/2)
    my_list = [1]*elements + [0]*elements
    my_list = my_list*(scenarios_by_step**(t))

    for s, element in enumerate(my_list):
        wind_high_param.update({(t+1, s+1):element})

wind_low_param = {}
for t in range(num_step):
    elements = int(scenarios_by_step**(num_step-t)/2)
    my_list = [0]*elements + [1]*elements
    my_list = my_list*(scenarios_by_step**(t))

    for s, element in enumerate(my_list):
        wind_low_param.update({(t+1, s+1):element})

data.update({'wind_high' : wind_high_param}) 
data.update({'wind_low' : wind_low_param}) 


prob_param = {}

single_scenario_probabilities = [high_wind_probability * high_demand_probability, 
                                high_wind_probability * medium_demand_probability,
                                high_wind_probability * low_demand_probability,
                                low_wind_probability  * high_demand_probability,
                                low_wind_probability  * medium_demand_probability,
                                low_wind_probability  * low_demand_probability]

all_probabilities = [A * B * C \
                     for A in single_scenario_probabilities
                     for B in single_scenario_probabilities
                     for C in single_scenario_probabilities]

for s, element in enumerate(all_probabilities):
    prob_param.update({s+1 : element})

data.update({'prob' : prob_param})



demand_param = {}
for t in range(num_step):

    current_demand_profile = demands[t+1]

    current_demand_profile = current_demand_profile*2

    multiply = ((scenarios_by_step)**(num_step-t-1))

    current_demand_profile = [item for item in current_demand_profile for _ in range(multiply)]

    groups = scenarios_by_step**t

    current_demand_profile = current_demand_profile*groups

    for s, element in enumerate(current_demand_profile):
        demand_param.update({(t+1,s+1) : element})

data.update({'demand' : demand_param}) # !!!!

data = {None : data}


### Objective and constraints

In [23]:


# Objective function
def ObjRule(cfa):
        return sum( 

            (
                cfa.nuc_act[s] * cfa.cost_activate_nuc + \
                sum(cfa.gas_act[t,s] * cfa.cost_activate_gas for t in cfa.T) + \
                sum(cfa.gas_qnt[t,s] * cfa.cost_kw_gas for t in cfa.T) + \
                sum(
                    (cfa.wind_high[t,s] * cfa.power_wind_high + cfa.wind_low[t,s] * cfa.power_wind_low )
                    * cfa.cost_kw_wind for t in cfa.T ) +\
                
                sum(cfa.ext_qnt[t,s] * cfa.cost_kw_ext for t in cfa.T)
            ) * cfa.prob[s]
        
            
            for s in cfa.S)


# Constraints

def maxPowerGasRule(cfa, t, s):
        return cfa.gas_qnt[t,s] <= cfa.max_power_gas
    
def activationGas(cfa, t, s):
        return cfa.gas_qnt[t,s] <= cfa.gas_act[t,s] * BIG_M

def satisfyDemand(cfa, t, s):
        return cfa.demand[t,s] <= cfa.power_nuc * cfa.nuc_act[s] + \
            cfa.gas_qnt[t,s] + \
            cfa.ext_qnt[t,s] + \
            cfa.wind_high[t,s] * cfa.power_wind_high + cfa.wind_low[t,s] * cfa.power_wind_low


### Model

In [24]:


cfa = pyo.AbstractModel() # pyomo abstract model

# Sets
cfa.S = pyo.Set(initialize = list(range(1, scenarios_by_step**num_step+1)))
cfa.S.construct()
cfa.T = pyo.Set(initialize = list(range(1, num_step+1)))
cfa.T.construct()
cfa.NonAnt = pyo.Set(cfa.T, initialize = non_ant_sets)
cfa.NonAnt.construct()


# Parameters
cfa.cost_activate_nuc = pyo.Param() # Activate nuclear in one step 
cfa.power_nuc = pyo.Param()    # Power provided from nuclear (constant)
        
cfa.cost_activate_gas = pyo.Param() # Activate gas in one step
cfa.cost_kw_gas = pyo.Param()  # Cost per kw from gas
cfa.max_power_gas = pyo.Param()# Max power from gas

cfa.cost_kw_wind = pyo.Param() # Cost per kw from wind
cfa.power_wind_high = pyo.Param()
cfa.power_wind_low = pyo.Param()

cfa.wind_high = pyo.Param(cfa.T, cfa.S) # 1 on wind high scenarions, 0 on wind low ones
cfa.wind_low = pyo.Param(cfa.T, cfa.S) # 1 on wind low scenarions, 0 on wind high ones

cfa.prob = pyo.Param(cfa.S)

cfa.cost_kw_ext = pyo.Param()  # Soft demand satisfaciton constraints

cfa.demand = pyo.Param(cfa.T, cfa.S)


# Variables
cfa.nuc_act = pyo.Var(cfa.S, within=pyo.Binary)
cfa.gas_act = pyo.Var(cfa.T, cfa.S, within=pyo.Binary)
cfa.gas_qnt = pyo.Var(cfa.T, cfa.S, within=pyo.NonNegativeReals)
cfa.ext_qnt = pyo.Var(cfa.T, cfa.S, within=pyo.NonNegativeReals)


# Objective function
cfa.obj = pyo.Objective(rule=ObjRule, sense=pyo.minimize)
        

# Constraints
cfa.maxPowerGas = pyo.Constraint(cfa.T, cfa.S, rule=maxPowerGasRule)
cfa.activationGas = pyo.Constraint(cfa.T, cfa.S, rule=activationGas)
cfa.satisfyDemand = pyo.Constraint(cfa.T, cfa.S, rule=satisfyDemand)



### Solve

In [25]:
instance = cfa.create_instance(data)

In [26]:
solver = pyo.SolverFactory('appsi_highs')
              
solver_result = solver.solve(instance, tee=False)
                
instance.solutions.store_to(solver_result)


In [27]:
instance.gas_qnt[2,85].value

5000.0